In [354]:
import pandas as pd
import numpy as np

In [355]:
#!ls *.csv
df = pd.read_csv("West_Nile_Virus_by_County.csv")
df.head(2)

,Year,Week_Reported,County,id,Positive_Cases
0,2006,35,Alameda,6001,1
1,2006,33,Butte,6007,4


In [356]:
counties=df["County"].unique()
counties

array(['Alameda', 'Butte', 'Colusa', 'Contra Costa', 'El Dorado',
       'Fresno', 'Glenn', 'Imperial', 'Kern', 'Kings', 'Lake',
       'Los Angeles', 'Marin', 'Merced', 'Modoc', 'Mono', 'Napa',
       'Nevada', 'Orange', 'Placer', 'Riverside', 'Sacramento',
       'San Bernardino', 'San Diego', 'San Joaquin', 'San Luis Obispo',
       'Santa Clara', 'Shasta', 'Solano', 'Stanislaus', 'Sutter',
       'Tehama', 'Tulare', 'Ventura', 'Yolo', 'Yuba', 'Madera',
       'Mendocino', 'Sonoma', 'Calaveras', 'Santa Barbara', 'Monterey',
       'San Francisco', 'Amador', 'Santa Cruz', 'Siskiyou'], dtype=object)

In [357]:
#!C:\Users\vmamidal\AppData\Local\Programs\Python\Python39\Scripts\pip.exe install geocoder
import geocoder
g = geocoder.osm(str(df['County'][0])+" County, CA")
#g.geojson
#g.json
#g.wkt
#g.osm
g.geojson['features'][0]['geometry']['coordinates']

[-121.899142, 37.6090291]

In [358]:
geocounties=[]
for x in counties:
    geocounties.append(geocoder.osm(str(x)+" County, CA").geojson['features'][0]['geometry']['coordinates'])
#geocounties

In [359]:
countiesdf=pd.DataFrame()
countiesdf['geoLocation']=geocounties
countiesdf=pd.DataFrame(countiesdf['geoLocation'].to_list(), columns=['Latitude','Longitude'])
countiesdf['County']=counties
countiesdf.head()

,Latitude,Longitude,County
0,-121.899142,37.609029,Alameda
1,-121.585844,39.651927,Butte
2,-122.220956,39.146558,Colusa
3,-121.917535,37.903481,Contra Costa
4,-120.527613,38.757414,El Dorado


In [361]:
DF=df.merge(countiesdf)
DF=DF.sort_values(["Year", "Week_Reported"])
DF.head()

,Year,Week_Reported,County,id,Positive_Cases,Latitude,Longitude
262,2006,25,Kern,6029,1,-118.753822,35.314570
263,2006,29,Kern,6029,1,-118.753822,35.314570
970,2006,29,San Joaquin,6077,1,-121.277372,37.937290
1089,2006,29,Stanislaus,6099,1,-121.050143,37.550087
1187,2006,29,Sutter,6101,1,-121.697088,38.950967


In [362]:
DFmapdata=DF[["Longitude", "Latitude"]].values
DFmapdata[0]

array([  35.3145701, -118.7538222])

In [363]:
#df['ID'] = df['id'].apply(lambda x: str('0500000US0')+str(x))
#df.head(2)

In [364]:
import os
import folium
import pandas as pd
from folium.plugins import HeatMap, HeatMapWithTime
from folium.plugins import MarkerCluster
from folium import IFrame
from folium import Marker

In [367]:
#ca;ifornia co ordinates
CA_COORDINATES = (36.7783, -119.4179)
map1 = folium.Map(location=CA_COORDINATES,tiles='OpenStreetMap', zoom_start=6)
HeatMap(DFmapdata).add_to(map1)
map1.save('Excercise7-Heatmap.html')
display(map1)

In [384]:
#https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/
# create empty map zoomed in on California
map2 = folium.Map(location=CA_COORDINATES,tiles="OpenStreetMap", zoom_start=5)
# add a marker for every record in the filtered data, use a clustered view
for each in DF[0:DF.shape[0]].iterrows():
    county = DF["County"].iloc[idx]
    cases = DF["Positive_Cases"].iloc[idx]
    week = DF["Week_Reported"].iloc[idx]
    year = DF["Year"].iloc[idx]
    folium.Marker(location = [each[1]['Longitude'],each[1]['Latitude']],clustered_marker = True,\
                  popup="County:{}\n Cases:{}\n Week:{}\n Year:{}".format(county,cases,week, year)).add_to(map2) 
map2.save('Excercise7-markers.html')
display(map2)

In [386]:
# create empty map zoomed in on California by cluster grouping.
map3 = folium.Map(location=CA_COORDINATES,tiles="OpenStreetMap", zoom_start=6)
for idx in range(DF.shape[0]):
    county = DF["County"].iloc[idx]
    cases = DF["Positive_Cases"].iloc[idx]
    week = DF["Week_Reported"].iloc[idx]
    year = DF["Year"].iloc[idx]
g = folium.FeatureGroup(name='West Nile Virus Cases')
g.add_child(MarkerCluster(locations=DFmapdata))
g.add_child(folium.Popup("County:{}\n Cases:{}\n Week:{}\n Year:{}".format(county,cases,week, year)))
map3.add_child(g)
map3.save('Excercise7-markers-Groups.html')
display(map3)